# Sea Surface Temperature

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from maps import map_robinson
from paths import path_results
from plotting import shifted_color_map
from timeseries import IterateOutputCESM
from xr_regression import xr_linear_trends_2D

In [ ]:
# stacking files into one xr Dataset object
for run in ['ctrl', 'rcp']:
    for i, (y,m,s) in enumerate(IterateOutputCESM('ocn_rect', run, 'yrly', name='TEMP_PD')):
        da = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
        if i==0:
            da_new = da.copy()
        else:
            da_new = xr.concat([da_new, da], dim='time')
    da_new.to_netcdf(f'{path_results}/SST/SST_yrly_{run}.nc')

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_results}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_results}/SST/SST_yrly_rcp.nc' )

In [ ]:
SST_yrly_ctrl

In [ ]:
SST_trend_ctrl = xr_linear_trends_2D(SST_yrly_ctrl, dim_names=('lat', 'lon'))
SST_trend_rcp  = xr_linear_trends_2D(SST_yrly_rcp , dim_names=('lat', 'lon'))

In [ ]:
SST_ctrl_mean = SST_yrly_ctrl.mean(dim='time')
fn = f'{path_results}/SST/SST_ctrl_mean'

f = map_robinson(xr_DataArray=SST_ctrl_mean, cmap='rainbow', minv=-2, maxv=30, label='SST [$^\circ$C]', filename=fn)

In [ ]:
SST_rcp_ctrl = (SST_yrly_rcp[-10:,:,:].mean(dim='time') - SST_yrly_ctrl.mean(dim='time'))
fn = f'{path_results}/SST/SST_last_rcp_decade_minus_ctrl_avg'
cmap = shifted_color_map(mpl.cm.RdBu_r, start=.25, midpoint=0.5, stop=1., name='shrunk')

f = map_robinson(xr_DataArray=SST_rcp_ctrl, cmap=cmap,minv=-2,maxv=4, label='$\Delta$SST [K]', filename=fn)